https://www.geeksforgeeks.org/stock-price-prediction-using-machine-learning-in-python/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics
 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#df = pd.read_csv('c:/data/^KS11.csv')

import pandas as pd
import yfinance as yf
symbol = 'MSFT'
period = '20y'
interval = '1d'


q = yf.Ticker(symbol)
df = q.history(period=period, interval=interval)

df.reset_index(inplace=True)

df = df.drop(['Dividends', 'Stock Splits', 'Capital Gains'], axis=1)

df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

splitted = df['Date'].str.split('-', expand=True)
 
df['day'] = splitted[2].astype('int')
df['month'] = splitted[1].astype('int')
df['year'] = splitted[0].astype('int')

print('shape', df.shape)

df.head()



In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df['Close'])
plt.title(symbol+' Close price.', fontsize=15)
plt.ylabel('Price in dollars.')
plt.show()

In [ ]:
features = ['Open', 'High', 'Low', 'Close', 'Volume']
 
plt.subplots(figsize=(20,10))
 
for i, col in enumerate(features):
  plt.subplot(2,3,i+1)
  sb.distplot(df[col])
plt.show()

In [ ]:
plt.subplots(figsize=(20,10))
for i, col in enumerate(features):
  plt.subplot(2,3,i+1)
  sb.boxplot(df[col])
plt.show()

In [ ]:
df['is_quarter_end'] = np.where(df['month']%3==0,1,0)
df.head()

In [ ]:
dd = df.drop(['Date'], axis=1)
data_grouped = dd.groupby('year').mean()
plt.subplots(figsize=(20,10))
 
for i, col in enumerate(['Open', 'High', 'Low', 'Close']):
  plt.subplot(2,2,i+1)
  data_grouped[col].plot.bar()
plt.show()

In [ ]:
dd.groupby('is_quarter_end').mean()

In [ ]:
df['open-close']  = df['Open'] - df['Close']
df['low-high']  = df['Low'] - df['High']
df['target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

In [ ]:
plt.pie(df['target'].value_counts().values,
        labels=[0, 1], autopct='%1.1f%%')
plt.show()

In [ ]:
dd = df.drop(['Date'], axis=1)

plt.figure(figsize=(10, 10))
 
# As our concern is with the highly
# correlated features only so, we will visualize
# our heatmap as per that criteria only.
sb.heatmap(dd.corr() > 0.9, annot=True, cbar=False)
plt.show()

In [ ]:
features = df[['open-close', 'low-high', 'is_quarter_end']]
target = df['target']
 
scaler = StandardScaler()
features = scaler.fit_transform(features)
 
X_train, X_valid, Y_train, Y_valid = train_test_split(
    features, target, test_size=0.1, random_state=2022)
print(X_train.shape, X_valid.shape)

In [ ]:
models = [LogisticRegression(), SVC(
  kernel='poly', probability=True), XGBClassifier()]
 
for i in range(3):
  models[i].fit(X_train, Y_train)
 
  print(f'{models[i]} : ')
  print('Training Accuracy : ', metrics.roc_auc_score(
    Y_train, models[i].predict_proba(X_train)[:,1]))
  print('Validation Accuracy : ', metrics.roc_auc_score(
    Y_valid, models[i].predict_proba(X_valid)[:,1]))
  print()

In [ ]:
metrics.ConfusionMatrixDisplay(models[0], X_valid, Y_valid)
plt.show()
#class sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix, *, display_labels=None)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
#X, y = make_classification(random_state=0)
#X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                    random_state=0)
clf = SVC(random_state=0)
clf.fit(X_train, Y_train)
SVC(random_state=0)
predictions = clf.predict(X_valid)
cm = confusion_matrix(Y_valid, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=clf.classes_)
disp.plot()

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import yfinance as yf
from datetime import time, date, datetime, timedelta
import pytz, holidays
from pytz import timezone
import numpy as np

import warnings

warnings.filterwarnings("ignore")

import sys

symbol='MSFT'

q = yf.Ticker(symbol)

current_price = q.history(period='1d', interval='1d').iloc[0]['Close']

max_calls = q.option_chain().calls
max_puts = q.option_chain().puts
#calls = q.option_chain(exp_date.strftime("%Y-%m-%d")).calls           
#puts = q.option_chain(exp_date.strftime("%Y-%m-%d")).puts     

#today = datetime.now(timezone(app_settings.TIMEZONE))                 
#days_to_expire = (exp_date.tz_localize(timezone(app_settings.TIMEZONE))-today).days
#calls[oq.DAYS_TO_EXPIRE] = puts[oq.DAYS_TO_EXPIRE] =  days_to_expire                
#calls[oq.EXP_DATE] =puts[oq.EXP_DATE] = exp_date.strftime("%Y-%m-%d")
#calls[oq.ANNUALIZED_RETURN] = puts[oq.ANNUALIZED_RETURN] = np.nan
#calls[oq.PNL] = puts[oq.PNL] = np.nan     
#calls[oq.PROFIT] = puts[oq.PROFIT] = np.nan      


max_calls['diff'] = max_calls.apply(lambda x: (current_price-x['strike'])/current_price, axis = 1)    
max_puts['diff'] = max_puts.apply(lambda x: (current_price-x['strike'])/current_price, axis = 1) 
max_calls['price'] = (max_calls['ask'] + max_calls['bid'])
max_puts['price'] = (max_puts['ask'] + max_puts['bid'])
max_calls['bid ask spread'] = (max_calls['ask'] + max_calls['bid'])
max_puts['bid ask spread'] = (max_puts['ask'] + max_puts['bid'])
                                            
max_calls                                

features = ['impliedVolatility', 'volume', 'openInterest', 'strike', 'price', 'bid ask spread', 'diff']
 
df = max_puts

plt.subplots(figsize=(20,10))
 
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(df[col])
plt.show()

plt.subplots(figsize=(20,10))
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.boxplot(df[col])
plt.show()

df = max_puts

plt.subplots(figsize=(20,10))
 
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(df[col])
plt.show()

plt.subplots(figsize=(20,10))
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.boxplot(df[col])
plt.show()

In [ ]:
total_puts['strike']

In [ ]:
import sys

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.utils.dataGetter import get_multiple_option_chains
from tradingBot.utils.dataGetter import pick_option_long
from tradingBot.utils.dataGetter import pick_option_short
from tradingBot.utils.dataGetter import get_price_history

In [ ]:
sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.utils.dataGetter import get_multiple_option_chains

symbol='MSFT'

total_calls, total_puts = get_multiple_option_chains(symbol, 
                                strike_range=1,
                                target_exp_date_list=[],                                 
                                max_days_to_expire=720, 
                                max_bid_ask_spread=10,
                                min_open_interest=100)   

#puts = puts.sort_values('price', ascending=False)
print('count= ',total_puts.shape[0])
total_puts['strike']

In [ ]:
current_price = get_price_history(symbol, period='1d', interval='1d').iloc[0]['Close']

current_price

In [ ]:
features = ['impliedVolatility', 'days to expire', 'strike', 'volume', 'openInterest', 'bid ask spread', 'win prob short', 'delta']
 
plt.subplots(figsize=(30,20))
 
df = total_calls

for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(df[col])
plt.show()

In [ ]:
from tradingBot.consts import asset as at
from tradingBot.consts import optionQuote as oq

calls=total_calls
puts=max_puts

target_price = current_price           

min_annualized_return=20
min_IV = 0.5
min_pnl = 0.25
max_bid_ask_spread=1.0
max_days_to_expire=90 
min_open_interest=100

i = puts.shape[0]
puts = puts[puts[oq.STRIKE] >= target_price]
print('%d -> %d put fillterd out by IV >= %.2f' % (i, puts.shape[0], min_IV))    
#logger.debug('%d -> %d put fillterd out by IV >= %.2f' % (i, puts.shape[0], min_IV))

    
#i = calls.shape[0]
#calls = calls[calls[oq.IMPLIED_VOLATILITY] >= min_IV]
#print('%d -> %d call fillterd out by IV >= %.2f' % (i, calls.shape[0], min_IV))
#logger.debug('%d -> %d call fillterd out by IV >= %.2f' % (i, calls.shape[0], min_IV))

In [ ]:
total_puts.sort_values('strike', ascending=False)[['strike', 'impliedVolatility' ]]

In [ ]:
total_puts.sort_values('strike', ascending=False)[['exp date', 'strike', 'price', 'delta','impliedVolatility' ]]

In [ ]:
'''

    i = puts.shape[0]
    puts = puts[puts[oq.IMPLIED_VOLATILITY] >= min_IV]
    #print('%d -> %d put fillterd out by IV >= %.2f' % (i, puts.shape[0], min_IV))    
    logger.debug('%d -> %d put fillterd out by IV >= %.2f' % (i, puts.shape[0], min_IV))

    calls[oq.PROFIT] = calls.apply(lambda x: x[oq.PRICE] if target_price < x[oq.STRIKE] else -10000, axis = 1)    
    puts[oq.PROFIT] = puts.apply(lambda x: x[oq.PRICE] if target_price > x[oq.STRIKE] else x[oq.PRICE] - x[oq.STRIKE] , axis = 1)    

    calls[oq.ANNUALIZED_RETURN] = calls.apply(lambda x: pow(1+(x[oq.PROFIT]/x[oq.STRIKE]), 52/(x[oq.DAYS_TO_EXPIRE]/7)).real-1, axis = 1)
    puts[oq.ANNUALIZED_RETURN] = puts.apply(lambda x: pow(1+(x[oq.PROFIT]/x[oq.STRIKE]), 52/(x[oq.DAYS_TO_EXPIRE]/7)).real-1, axis = 1)

    calls[oq.PNL] = calls.apply(lambda x: x[oq.PROFIT]/x[oq.PRICE], axis = 1)    
    puts[oq.PNL] = puts.apply(lambda x: x[oq.PROFIT]/x[oq.PRICE], axis = 1)    
 
    i = calls.shape[0]
    calls = calls[calls[oq.PNL] >= min_pnl]
    #print('%d -> %d call fillterd out by PNL >= %.2f' % (i, calls.shape[0], min_pnl))    
    logger.debug('%d -> %d call fillterd out by PNL >= %.2f' % (i, calls.shape[0], min_pnl))

    i = puts.shape[0]
    #print('%d -> %d puts fillterd out by PNL >= %.2f' % (i, puts.shape[0], min_pnl))      
    logger.debug('%d -> %d puts fillterd out by PNL >= %.2f' % (i, puts.shape[0], min_pnl))

    calls = calls.sort_values(oq.PNL)
    puts = puts.sort_values(oq.PNL)

    return calls, puts

'''

In [ ]:
i

In [ ]:
p = total_puts[total_puts['exp date'] == '2023-06-02']
p = p.sort_values('strike')
p[['exp date', 'strike', 'lastPrice', 'price', 'delta', 'theta','gamma','vega', 'impliedVolatility', 'win prob short' ]]

In [ ]:
from tradingBot.utils.dataGetter import pick_option_long
from tradingBot.utils.dataGetter import pick_option_short
from tradingBot.utils.dataGetter import get_price_history

symbol = 'MSFT'

current_price = get_price_history(symbol, period='1d', interval='1d').iloc[0]['Close']

target_price = current_price

calls, lputs = pick_option_long(symbol, 
                                target_price,            
                                min_annualized_return=20,
                                min_pnl = 0.25,
                                max_IV = 0.3,
                                max_bid_ask_spread=5,
                                max_days_to_expire=90, 
                                min_open_interest=100)

In [ ]:
#calls.sort_values('delta')[['exp date', 'strike', 'delta']]
calls

In [ ]:
from tradingBot.utils.dataGetter import pick_option_long
from tradingBot.utils.dataGetter import pick_option_short
from tradingBot.utils.dataGetter import get_price_history

symbol = 'MSFT'

current_price = get_price_history(symbol, period='1d', interval='1d').iloc[0]['Close']

target_price = current_price * 1.0

calls, sputs = pick_option_short(symbol, 
                                target_price,            
                                min_annualized_return=20,
                                min_pnl = 0.25,
                                min_IV = 0.3,
                                max_bid_ask_spread=5,
                                max_days_to_expire=90, 
                                min_open_interest=100)

print(current_price)

In [ ]:
sputs.sort_values('price')[['exp date', 'strike', 'price', 'delta']]

In [ ]:
calls

In [ ]:
import sys
sys.path.append(r'\Users\jimhu\option_trader\src')

from tradingBot.utils.dataGetter import pick_vertical_put_spreads
from tradingBot.utils.dataGetter import get_price_history

symbol = 'MSFT'
current_price = get_price_history(symbol, period='1d', interval='1d').iloc[0]['Close']
target_price = current_price * 0.9

target_high_price = current_price * 1
target_low_price = current_price * 1

spread_list = pick_vertical_put_spreads(symbol, target_high_price, target_low_price,
                                        credit=True,
                                        target_exp_date_list=[],
                                        target_spread_list=[],
                                        min_pnl=0.8,
                                        min_short_IV=0.1,
                                        max_long_IV=0.9,
                                        min_win_prob=50,
                                        min_annualized_return=25,
                                        max_bid_ask_spread=2.0,
                                        max_days_to_expire=90, 
                                        min_open_interest=100)

print(spread_list)

In [ ]:
len(spread_list)

In [ ]:
from tradingBot.consts import asset as at
from tradingBot.utils.optionTool import delta_call
from tradingBot.utils.optionTool import delta_put
from tradingBot.utils.optionTool import time_to_maturity_in_year

trade_date = datetime.now().astimezone(timezone(app_settings.TIMEZONE)).date()

stock_price = get_price_history(symbol, period='1d', interval='1d').iloc[0]['Close']


In [ ]:
stock_price

In [ ]:
exp_date='2023-06-02'
time_to_maturity_in_year(trade_date, exp_date)

In [ ]:
exp_date='2023-06-02'

T = time_to_maturity_in_year(trade_date, exp_date)
print(T)
strike=300

print('call delta=', delta_call(stock_price, strike, T))
    
print('put delta=', delta_put(stock_price, strike, T))
    

In [ ]:
price0=stock_price
vol=0.25 
interest=0.05
dividend=0.02    
Time=T

In [ ]:
from scipy.stats import norm
import numpy as np

def black_scholes_greeks(S, K, T, r, sigma, option = 'call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option == 'call':
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
    elif option == 'put':
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    
    return delta, gamma, theta, vega

In [ ]:
exp_date='2023-06-02'

trade_date = datetime.now().astimezone(timezone(app_settings.TIMEZONE)).date()

stock_price = get_price_history(symbol, period='1d', interval='1d').iloc[0]['Close']


strike=300

print('call delta=', delta_call(stock_price, strike, T))
    
print('put delta=', delta_put(stock_price, strike, T))

S = stock_price
K = strike
T = time_to_maturity_in_year(trade_date, exp_date)
print(T)
sigma=0.25
r=0.05

black_scholes_greeks(S, K, T, r, sigma)